In [2]:
import numpy as np
import pandas as pd
import re
import nltk 
from sklearn.datasets import load_files
nltk.download('stopwords')
import pickle
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import textstat as ts
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jrnoo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [115]:
def mreplace(text,list1,list2): #replaces strings in text more efficiently than built-in methods
    for i in range(len(list1)):
        text=text.replace(list1[i],list2[i])
    return text

def remover(text,removees): #removes a list of strings from text
    nothing = [ "" for i in removees ]
    return mreplace(text,removees,nothing)

def plaintxt(text,nospace=True): #removes all spaces, line breaks, and punctuation from text
    removees = ["\n",".",",","?","(",")","!","/",":",";","-","'",'"',"[","]"]
    if nospace: # If nospace is false, spaces will not be removed
        removees.append(" ")
    return remover(text,removees)

def sentlength(text): #average sentence length
    return ts.lexicon_count(text)/ts.sentence_count(text)

def wordlength(text): #average letters per word
    return len(plaintxt(text))/ts.lexicon_count(text)

def wordsyll(text): #average syllables per word
    return ts.syllable_count(text)/ts.lexicon_count(text)

In [5]:
allSW = set(stopwords.words('english')) #set of all stop words

def stopwordprop(text): #Proportion of stop words in text
    nopunc = plaintxt(text,nospace=False) #get rid of punctuations and line breaks
    words = nopunc.lower().split() #remove capitalization and split into individual words
    count = 0
    for word in words: #count the number of stop words in the text
        if word in allSW:
            count += 1 
    return count/len(words)

In [9]:
def phrases(text,n):
    nopunc = plaintxt(text,nospace=False) #probably want to add something here to deal with sentences ending, but
                                          # I'll do that later
    words = nopunc.lower().split()
    phraselist = [" ".join(words[i:i+n]) for i in range(len(words)-n+1)] #I might need to change " " to "" when I 
                                            # use this with the word vectorizer
    return phraselist

In [55]:
def phrase_count(text,n=1,minnum=0):
    ngrams = phrases(text,n)
    unique_phrases = np.unique(np.array(ngrams))
    counter = {}
    for gram in unique_phrases:
        num = ngrams.count(gram)
        if num>=minnum: counter[gram] = num
    return counter

In [90]:
def dataframeizer(functions): # turns a list of functions that take in text into a series of functions that take in data frames
    funcs = pd.Series(functions) #starts by turning the list into a series
    newfunc = lambda f: lambda df: df.apply(f) #second-order function that takes in a function f and returns a function that applies f to a data frame or series
    newfs = funcs.apply(newfunc) #turns every function in funcs from a text function to a data frame function
    return newfs

In [58]:
RL = lambda text: ts.text_standard(text,float_output=True) # Reading level of the text

In [103]:
sentlen, wordlen, avesylls, SWprop, readlvl, sentcount, wordcount = dataframeizer([sentlength,wordlength,wordsyll,stopwordprop,ts.flesch_reading_ease,ts.sentence_count,ts.lexicon_count])

In [108]:
def Ngrams(df,n,mini=0):
    ngcount = lambda text: phrase_count(text,n=n,minnum=mini)
    return df.apply(ngcount)